In [25]:
import os

import numpy as np
from tqdm import tqdm

np.random.seed(0)
import json
from functools import partial

import cv2
import h5py
import joblib
import numpy as np
from skimage import data, feature, future
from skimage.io import imread, imsave
from skimage.transform import rescale, resize
from sklearn.ensemble import RandomForestClassifier

In [37]:
run_name = 1
annotations_paths = [
    f"annotation_slices/MultiMosaic/Run_{x}/annotations.json"
    for x in range(1, run_name + 1)
]
all_label = []
all_images = []
downsample_xy = 0.25
h5_file = True
sigma_min = 1
sigma_max = 256 * downsample_xy

In [38]:
# Feature extraction CPU
features_func = partial(
    feature.multiscale_basic_features,
    intensity=True,
    edges=True,
    texture=True,
    sigma_min=sigma_min,
    sigma_max=sigma_max,
)

In [39]:
for annotations_path in annotations_paths:
    annotations_path = annotations_path.replace(".json", "_new.json")
    f = open(annotations_path)
    data = json.load(f)
    time_ind_old = -1
    for slice_annot in tqdm(data):
        split_name = slice_annot.split(".")[0].split("_")
        slice_ind = int(split_name[5])
        time_ind = int(split_name[3])
        position = int(split_name[1])

        if h5_file:
            stack_file = h5py.File(
                f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{str(position)}_Settings_1_Processed/denoised_registered_processed.h5",
                "r",
            )
            stack_cherry = (
                stack_file[f"t{time_ind:05}"]["s00"]["0"]["cells"][slice_ind]
                .astype(np.float32)
                .copy()
            )
            stack_gfp = (
                stack_file[f"t{time_ind:05}"]["s01"]["0"]["cells"][slice_ind]
                .astype(np.float32)
                .copy()
            )
            img = stack_cherry + stack_gfp

        img = rescale(img, [downsample_xy, downsample_xy], order=1, preserve_range=True)

        training_labels = np.zeros(np.array(img.shape), dtype=np.uint8)
        bboxes = data[slice_annot]
        for bbox in bboxes:
            if bbox["label"] == "background":
                training_labels[
                    int(downsample_xy * bbox["y"]) : (
                        int(downsample_xy * bbox["y"])
                        + int(downsample_xy * bbox["height"])
                    ),
                    int(downsample_xy * bbox["x"]) : (
                        int(downsample_xy * bbox["x"])
                        + int(downsample_xy * bbox["width"])
                    ),
                ] = 1
            if bbox["label"] == "organoid":
                training_labels[
                    int(downsample_xy * bbox["y"]) : (
                        int(downsample_xy * bbox["y"])
                        + int(downsample_xy * bbox["height"])
                    ),
                    int(downsample_xy * bbox["x"]) : (
                        int(downsample_xy * bbox["x"])
                        + int(downsample_xy * bbox["width"])
                    ),
                ] = 2
            if bbox["label"] == "lumen":
                training_labels[
                    int(downsample_xy * bbox["y"]) : (
                        int(downsample_xy * bbox["y"])
                        + int(downsample_xy * bbox["height"])
                    ),
                    int(downsample_xy * bbox["x"]) : (
                        int(downsample_xy * bbox["x"])
                        + int(downsample_xy * bbox["width"])
                    ),
                ] = 3
        # print(np.unique(training_labels))
        # plt.imshow(img,cmap='gray')
        # plt.imshow(training_labels, cmap='hot', alpha=0.5)
        # plt.show()

        # features = features_func(cp.asarray(img)).get()
        features = features_func(img)
        features = features.reshape(-1, features.shape[-1])
        features = features[training_labels.flatten() != 0]
        training_labels = training_labels.flatten()[training_labels.flatten() != 0]
        time_ind_old = time_ind
        all_images.append(features)
        all_label.append(training_labels)

100%|██████████| 5/5 [00:02<00:00,  1.67it/s]


In [48]:
X_all = np.vstack(all_images)
y_all = np.hstack(all_label)
# Remove duplicates
duplicates_indices = np.unique(X_all, axis=0, return_index=True)[1]
X = X_all[duplicates_indices]
y = y_all[duplicates_indices]

In [ ]:
clf = RandomForestClassifier(
    n_estimators=50,
    n_jobs=-1,
    max_depth=25,
    max_samples=0.05,
    verbose=1,
    random_state=42,
)
clf = future.fit_segmenter(y, X, clf)

In [ ]:
# saveLook
rf_location = "/cluster/home/gutgi/git_repositories/morphodynamics-of-human-brain-organoid-patterning/light_sheet_analysis/tissue_segmentation/rf_classifier/"
if not os.path.exists(rf_location):
    os.makedirs(rf_location)

In [ ]:
joblib.dump(clf, rf_location + "lumen_multimosaic_v15_05.joblib")

In [ ]:
# load
clf = joblib.load(rf_location + "lumen_multimosaic_v15_05.joblib")

In [ ]:
annotation_run = 44
run_name = f"Run_{annotation_run}"
output_dir = f"annotation_slices/MultiMosaic/{run_name}/"

In [ ]:
n_slices = 10
positions = np.random.choice(
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], size=n_slices
)
random_stack_names = np.random.choice(np.arange(1, 124), size=n_slices)
random_slice_indices = np.random.choice(np.arange(0, 231), size=n_slices)
all_positions = [positions, random_stack_names, random_slice_indices]
all_positions = np.unique(np.stack(all_positions, axis=1), axis=0)

for position, random_stack_name, random_slice_indice in tqdm(all_positions):
    if h5_file:
        stack_file = h5py.File(
            f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{str(position)}_Settings_1_Processed/denoised_registered_processed.h5",
            "r",
        )
        random_stack_cherry = (
            stack_file[f"t{random_stack_name:05}"]["s00"]["0"]["cells"][
                random_slice_indice
            ]
            .astype(np.float32)
            .copy()
        )
        random_stack_gfp = (
            stack_file[f"t{random_stack_name:05}"]["s01"]["0"]["cells"][
                random_slice_indice
            ]
            .astype(np.float32)
            .copy()
        )
        random_slice = random_stack_cherry + random_stack_gfp
        random_slice_2 = random_slice.copy()

        random_slice *= 255.0 / np.percentile(random_slice.copy(), q=99.5)

    else:
        random_stack_cherry = imread(
            input_dir
            + f"Position_{position}_Settings_1/denoised/mCherry/t{random_stack_name:04}_mCherry.tif"
        ).astype(np.float32)
        random_stack_gfp = imread(
            input_dir
            + f"Position_{position}_Settings_1/denoised/GFP/t{random_stack_name:04}_GFP.tif"
        ).astype(np.float32)

        random_stack = random_stack_cherry + random_stack_gfp
        random_slice = random_stack[random_slice_indice].copy()

        random_slice_2 = random_slice.copy()
        random_slice *= 255.0 / np.percentile(random_stack.copy(), q=99.5)

    random_slice_2 = rescale(
        random_slice_2, [downsample_xy, downsample_xy], order=1, preserve_range=True
    )
    random_slice = random_slice.clip(0, 255)
    image = np.stack((random_slice.astype(np.uint8),) * 3, axis=-1)

    features = features_func(random_slice_2)
    random_mask_slice = future.predict_segmenter(features, clf)
    # random_mask_slice=postprocess_masks(random_mask_slice,min_area=128)
    # random_mask_slice=smooth_organoids(random_mask_slice,2)
    random_mask_slice = resize(
        random_mask_slice, image.shape[:2], order=0, preserve_range=True
    ).astype("uint8")
    # combine
    colormask_organoid = np.zeros(image.shape, image.dtype)
    colormask_organoid[:, :] = (127, 255, 0)
    colormask_organoid = cv2.bitwise_and(
        colormask_organoid,
        colormask_organoid,
        mask=(random_mask_slice == 2).astype(np.uint8),
    )
    image = cv2.addWeighted(colormask_organoid, 0.05, image, 1, 0, image)

    colormask_lumen = np.zeros(image.shape, image.dtype)
    colormask_lumen[:, :] = (255, 99, 71)
    colormask_lumen = cv2.bitwise_and(
        colormask_lumen, colormask_lumen, mask=(random_mask_slice == 3).astype(np.uint8)
    )
    image = cv2.addWeighted(colormask_lumen, 0.1, image, 1, 0, image)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    imsave(
        f"{output_dir}position_{position}_stack_{str(random_stack_name)}_slice_{str(random_slice_indice)}.jpg",
        image,
    )